In [14]:
pip install pyvi

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install https://github.com/trungtv/vi_spacy/raw/master/packages/vi_spacy_model-0.2.1/dist/vi_spacy_model-0.2.1.tar.gz

  Created wheel for vi-spacy-model: filename=vi_spacy_model-0.2.1-py3-none-any.whl size=42371403 sha256=68c627f1e1bcc1df8ac4dff2311b6edd62382080c511b82c88cf8b6ad5297ff3
  Stored in directory: c:\users\nguye\appdata\local\pip\cache\wheels\db\6b\74\5ab06fee3cb3867afbeef75fca3a7a20556ddb28f8a100b8f8
Successfully built vi-spacy-model
Note: you may need to restart the kernel to use updated packages.


In [1]:
#!/usr/bin/env python
# coding: utf8
"""Example of training an additional entity type

This script shows how to add a new entity type to an existing pre-trained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.

The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.

After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.1.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals
from __future__ import print_function
import re
import os
import plac
import random
from pathlib import Path
import spacy
import json
import logging


ModuleNotFoundError: No module named 'plac'

In [86]:
traindata_file = "traindata_1.json"
model="vi_spacy_model"
new_model_name="training1"
output_dir='/output'
n_iter=20

In [87]:

# new entity label
LABEL = "COL_NAME"

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting

# training data
# TRAIN_DATA = [
#     ("i study in maria college", {"entities": [(11, 24, LABEL)]}),
#     ("completed graduation from napier university (edinburgh,
#       united kingdom)", {"entities": [(26, 43, LABEL)]}),
#     ("studied in school of continuing and professional studies",
#       {"entities": [(11, 16, LABEL)]}),
#     ("studied at chinese university of hong kong", {"entities":
#       [(11, 29, LABEL)]}),
#     ("studied in University of Strathclyde", {"entities":
#       [(11, 37, LABEL)]}),
# ]


def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(dataturks_JSON_FilePath, 'r', encoding="utf8") as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)

            text = data['content']
            entities = []
            if data['annotation'] is not None:
                for annotation in data['annotation']:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        # dataturks indices are both inclusive [start, end]
                        # but spacy is not [start, end)
                        entities.append((
                            point['start'],
                            point['end'], # + 1,
                            label
                        ))

            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception:
        logging.exception("Unable to process " + dataturks_JSON_FilePath)
        return None

SPACY_DATA = convert_dataturks_to_spacy(traindata_file)
print(SPACY_DATA)


[('Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 4,7 Tỷ Điện tích 38 m² Số tầng 5 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m', {'entities': [(0, 22, 'type'), (23, 51, 'saller'), (52, 62, 'price'), (63, 78, 'area'), (79, 88, 'floors'), (89, 99, 'rooms'), (100, 107, 'date'), (108, 118, 'phone'), (119, 131, 'front')]}), ('Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 5,8 Tỷ Điện tích 54 m² Số tầng 5 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m', {'entities': [(0, 22, 'type'), (23, 51, 'saller'), (52, 62, 'price'), (63, 78, 'area'), (79, 88, 'floors'), (89, 99, 'rooms'), (100, 107, 'date'), (108, 118, 'phone'), (119, 131, 'front')]}), ('Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 5,5 Tỷ Điện tích 80 m² Số tầng 1 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m', {'entities': [(0, 22, 'type'), (23, 51, 'saller'), (52, 62, 'price'), (63, 78, 'area'), (79, 88, 'floors'), (89, 99, 'rooms'), (100, 107, 'date'), (108, 118, 'phone'), (119, 131, 'front')]}), ('Loại tin Bán 

In [88]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
#     invalid_span_tokens = re.compile(r'\s')

    cleaned_data = data # []
#     for text, annotations in data:
#         entities = annotations['entities']
#         valid_entities = []
# #         print(text)
#         for start, end, label in entities:
#             valid_start = start
#             valid_end = end
#             while valid_start < len(text) and invalid_span_tokens.match(
#                     text[valid_start]):
# #                 print(valid_start, " -> ",text[valid_start])
#                 valid_start += 1

#             while valid_end > 1 and invalid_span_tokens.match(
#                     text[valid_end - 1]):
# #                 print(text[valid_end - 1])
#                 valid_end -= 1
#             valid_entities.append([valid_start, valid_end, label])
#         cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data
TRAIN_DATA = trim_entity_spans(SPACY_DATA)
print(TRAIN_DATA)


[('Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 4,7 Tỷ Điện tích 38 m² Số tầng 5 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m', {'entities': [(0, 22, 'type'), (23, 51, 'saller'), (52, 62, 'price'), (63, 78, 'area'), (79, 88, 'floors'), (89, 99, 'rooms'), (100, 107, 'date'), (108, 118, 'phone'), (119, 131, 'front')]}), ('Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 5,8 Tỷ Điện tích 54 m² Số tầng 5 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m', {'entities': [(0, 22, 'type'), (23, 51, 'saller'), (52, 62, 'price'), (63, 78, 'area'), (79, 88, 'floors'), (89, 99, 'rooms'), (100, 107, 'date'), (108, 118, 'phone'), (119, 131, 'front')]}), ('Loại tin Bán nhà riêng Tên liên hệ: Phan Thanh Tùng Giá 5,5 Tỷ Điện tích 80 m² Số tầng 1 Số phòng 0 hôm nay 0912142902 Mặt tiền 0 m', {'entities': [(0, 22, 'type'), (23, 51, 'saller'), (52, 62, 'price'), (63, 78, 'area'), (79, 88, 'floors'), (89, 99, 'rooms'), (100, 107, 'date'), (108, 118, 'phone'), (119, 131, 'front')]}), ('Loại tin Bán 

In [89]:
"""Set up the pipeline and entity recognizer, and train the new entity."""
random.seed(0)
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")
# Add entity recognizer to model if it's not in the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy

if "ner" not in nlp.pipe_names:
    print("Creating new pipe")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
else:
    # otherwise, get it, so we can add labels to it
    ner = nlp.get_pipe("ner")

Loaded model 'vi_spacy_model'
Creating new pipe


In [90]:

# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# if model is None or reset_weights:
#     optimizer = nlp.begin_training()
# else:
#     optimizer = nlp.resume_training()
move_names = list(ner.move_names)
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    # batch up the examples using spaCy's minibatch
    for itn in range(n_iter):
        print("Starting iteration " + str(itn))
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.1,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print("Losses", losses)
        


Starting iteration 0
Losses {'ner': 1873.470270335852}
Starting iteration 1
Losses {'ner': 183.01037692040705}
Starting iteration 2
Losses {'ner': 251.54994651519064}
Starting iteration 3
Losses {'ner': 137.466086199815}
Starting iteration 4
Losses {'ner': 339.1953427810109}
Starting iteration 5
Losses {'ner': 275.67223348366707}
Starting iteration 6
Losses {'ner': 167.67101491728442}
Starting iteration 7
Losses {'ner': 129.53831272693435}
Starting iteration 8
Losses {'ner': 118.14215338133694}
Starting iteration 9
Losses {'ner': 188.24659753705564}
Starting iteration 10
Losses {'ner': 104.77991220072234}
Starting iteration 11
Losses {'ner': 150.41348176123208}
Starting iteration 12
Losses {'ner': 111.44703957071413}
Starting iteration 13
Losses {'ner': 100.5031218114236}
Starting iteration 14
Losses {'ner': 43.009391164410026}
Starting iteration 15
Losses {'ner': 57.48550580822248}
Starting iteration 16
Losses {'ner': 279.21583070552043}
Starting iteration 17
Losses {'ner': 148.497240

In [91]:
def datafilter(label, value):
    if "tên liên_hệ" in value.lower():
        return value
    if "diện_tích" in value.lower():
        return value
    if "mặt_tiền" in value.lower():
        return value
    if "giá" in value.lower():
        return value
    
    
    return value

In [84]:
# print("Entities in '%s'" % test_text)
test_text = """BÁN GẤP NHÀ PHỐ ĐÀO TẤN, DIỆN TÍCH 50M2, XÂY 4 TẦNG, MẶT TIỀN 5.5M, GIÁ 5.3 Khu Vực: Bán Nhà Riêng Tại Đường Đào Tấn - Phường Cống Vị - Ba Đình - Hà Nội Giá: 5,3 Tỷ  Diện tích: 50 m² hôm nay Thông tin mô tả + Nhà nằm trong ngõ 12 Đào Tấn ô tô tránh có vỉa hè, ngõ thông nhiều phố, nhà cách ô tô đúng 20m, nhà vẫn còn rất mới, khách mua về là vào ở được ngay. + Nhà mới xây được mấy năm nhưng còn rất chắc chắn. Thiết kế nhà bê tông cốt thép chắc chắn, thiết kế hiện đại, nhà bao gồm: phòng khách, bếp, 04 phòng ngủ, phòng thờ, sân phơi + Sổ đỏ chính chủ, pháp lý rõ ràng, sẵn sang giao dịch + Giá 5.3 tỷ, có thương lượng cho khách có thiện trí Liên Hệ: Thanh Tùng BĐS Ba Đình: 0912142902, Quý khách gọi ngay để được tư vấn nhiệt tình và xem nhà miễn phí Cần Bán, ở ngay, ngõ thoáng, nhà còn mới, Ngõ thông, Đào Tấn, Ba Đình Liên hệ: 09121*** hiện số Loại tin rao Bán nhà riêng Pháp lý Sổ Hồng Số phòng 0 Đường vào 0 m Mặt tiền 0 m Số tầng 4 Số toilet 0 Thông tin liên hệ Tên liên hệ: Phan Thanh Tùng Loại tin: Môi giới Địa chỉ: Ba Đình Tỉnh thành Hà Nội Điện thoại: 0912142902 Email: thanhtungks68@gmail.com"""

test_text = """Bán căn hộ 312 đầu hồi nhà k6 đt việt hưng diện tích 98m, 3 phòng ngủ 2wc
Khu Vực: Bán Căn Hộ Chung Cư Tại Phường Việt Hưng - Long Biên - Hà Nội
Giá: 1,999 Tỷ Diện tích: 98 m²ID: 106954 | hôm kia|  48
Thông tin mô tả
Đô thị Việt Hưng, Long Biên, Hà Nội được ví như nhà ở giữ công viên.
- Ai ở đây hoặc có bạn bè, đến chơi, đi qua rồi thì biết ạ, chất lượng không khí, phong thủy, đất lành, vượng khí bao trùm, yên tĩnh ... là một nơi để ở, để nghỉ ngơi, nghỉ dưỡng sau những giờ làm việc mệt mỏi tại cơ quan.
- Là đô thị trung tâm của quận nên mọi cơ quan hành trính công đều nằm trong đô thị, cách nhà 200m đi bộ, cách 3 chợ Diêm Gỗ, kim Quan, Giang Biên 300m đi bộ. trường tiểu học, trung học, mầm non thì bên kia đường. bạn bè anh em đi xe oto đến chơi ăn nhậu dộ xe thỏa mái cả dưới lòng đường.
- Là đô thị có nhiều người gốc HÀ Nội sinh sống nên ẩm thực vô cùng phong phú và ngon khỏi chê, khuân viên dưới nhà rộng con em thoải mái vui chơi các môn thể thao, người dân thì thân thiện.
- Nói chung là một nơi đáng để ở. Gia đình không muốn bán đâu vì nội thất đồ dùng làm toàn bàng đồ xin, ghỗ tự nhiên hoàn toàn. Nhưng do chuyển công tác xa nên muốn bán lại cho ai thi??

Loại tin rao	Bán căn hộ chung cư
Pháp lý	Sổ Đỏ
Hướng nhà	Đông-Nam
Số phòng	3
Số tầng	3
Số toilet	2
Thông tin liên hệ
Tên liên hệ:	Đỗ Quốc Vương
Loại tin:	Chính chủ
Địa chỉ:	Long Biên
Tỉnh thành	Hà Nội
Điện thoại:	0971294648
Email:	vuonginox.sp@gmail.com"""
test_text = test_text.replace("\t"," ").replace("\n"," ").strip()

doc = nlp(test_text)
for ent in doc.ents:
    ent = datafilter(ent.label_,ent.text)
    if len(ent) > 1:
        print(ent)

# save model to output directory
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta["name"] = new_model_name  # rename model
with nlp.disable_pipes('ner'):
    nlp.to_disk(os.path.join(output_dir, new_model_name))
    print("Saved model to", output_dir)



diện_tích 98m
Giá: 1,999
Diện_tích:
Tên liên_hệ: Đỗ_Quốc_Vương
Saved model to \output


In [92]:
# test the saved model
test_text = """Nhà Đẹp phố Đội Cấn, diện tích 38m2, xây 05 tầng, mặt tiền 4.1m, giá 4.7 t\nLoại tin Bán nhà riêng\nVị trí nhà cách 20m ra mặt ngõ lớn 279 Đội Cấn, đường ô tô tránh, cả ngõ có 5-6 nhà. Tương lai mở rộng ngõ 279 nhà cách ngõ vài mét ( sau này ngõ 279 thành phố Đại Yên) giá trị nhà tăng chóng mặt. Diện tích 38m2, xây 05 tầng, mặt tiền rộng 4,1m, nhà còn rất mới, phù hợp cho việc vừa ở vừa kinh doanh, hoặc cho thuê văn phòng.Nhà chủ nhà tự xây, khung BTCT chắc chắn, thiết kế hiện đại+Tầng 1: 1 Phòng rộng + bếp + vs.+ Tầng 2: P khách + 1 ngủ + vs.+ Tầng 3+4: Mỗi tầng 2 ngủ + vs.+ Tầng 5: P. Thờ + sân phơi.Sổ đỏ chính chủ, pháp lý rõ ràng, sẵn sang giao dịchGiá: 4.7 tỷ có thương lượng cho khách có thiện trí.Liên Hệ: Thanh Tùng: 0912142902. Quý khách gọi ngay để được tư vấn nhiệt tình và xem nhà miễn phí. Nhà mới, ở ngay, ngõ nông, kinh doanh, cho thuê của hàng, văn phòng, Đội Cấn, Ba Đình  Liên hệ:  0912 142 90209121*** hiện số\nTên liên hệ: Phan Thanh Tùng\nGiá 4,7 Tỷ\nBán nhà riêng tại Đường Đội Cấn - Phường Đội Cấn -  Ba Đình -  Hà Nôi\nĐiện tích 38 m²\nSố tầng 5\nSố phòng 0\nhôm nay\n0912142902\nMặt tiền 0 m"""
test_text = test_text.replace("\n"," ")

test_text = """BÁN GẤP NHÀ PHỐ ĐÀO TẤN, DIỆN TÍCH 50M2, XÂY 4 TẦNG, MẶT TIỀN 5.5M, GIÁ 5.3 Khu Vực: Bán Nhà Riêng Tại Đường Đào Tấn - Phường Cống Vị - Ba Đình - Hà Nội Giá: 5,3 Tỷ  Diện tích: 50 m² hôm nay Thông tin mô tả + Nhà nằm trong ngõ 12 Đào Tấn ô tô tránh có vỉa hè, ngõ thông nhiều phố, nhà cách ô tô đúng 20m, nhà vẫn còn rất mới, khách mua về là vào ở được ngay. + Nhà mới xây được mấy năm nhưng còn rất chắc chắn. Thiết kế nhà bê tông cốt thép chắc chắn, thiết kế hiện đại, nhà bao gồm: phòng khách, bếp, 04 phòng ngủ, phòng thờ, sân phơi + Sổ đỏ chính chủ, pháp lý rõ ràng, sẵn sang giao dịch + Giá 5.3 tỷ, có thương lượng cho khách có thiện trí Liên Hệ: Thanh Tùng BĐS Ba Đình: 0912142902, Quý khách gọi ngay để được tư vấn nhiệt tình và xem nhà miễn phí Cần Bán, ở ngay, ngõ thoáng, nhà còn mới, Ngõ thông, Đào Tấn, Ba Đình Liên hệ: 09121*** hiện số Loại tin rao Bán nhà riêng Pháp lý Sổ Hồng Số phòng 0 Đường vào 0 m Mặt tiền 0 m Số tầng 4 Số toilet 0 Thông tin liên hệ Tên liên hệ: Phan Thanh Tùng Loại tin: Môi giới Địa chỉ: Ba Đình Tỉnh thành Hà Nội Điện thoại: 0912142902 Email: thanhtungks68@gmail.com"""

print("Loading from ", output_dir)
nlp2 = spacy.load(os.path.join(output_dir, new_model_name))    
ner = nlp.get_pipe("ner")
nlp2.add_pipe(ner)                  
# Check the classes have loaded back consistently
#     print(nlp2.get_pipe("ner").move_names,"\n",move_names)
#     assert nlp2.get_pipe("ner").move_names == move_names
doc2 = nlp2(test_text)


for ent in doc2.ents:
#     if (ent.label_ == "area"):
    ent = datafilter(ent.label_,ent.text)
    if len(ent) > 1:
        print(ent)
    


Loading from  /output
BÁN
GẤP NHÀ PHỐ ĐÀO TẤN, DIỆN_TÍCH
, XÂY 4 TẦNG, MẶT_TIỀN 5.5M, GIÁ 5.3 Khu_Vực: Bán Nhà_Riêng Tại Đường Đào_Tấn - Phường Cống_Vị - Ba_Đình - Hà_Nội Giá: 5,3 Tỷ Diện_tích: 50 m² hôm_nay Thông_tin mô_tả + Nhà nằm trong ngõ 12 Đào_Tấn ô_tô tránh có vỉa_hè, ngõ thông nhiều phố, nhà cách ô_tô đúng 20m, nhà vẫn còn rất mới, khách mua về là vào ở được ngay. + Nhà mới xây được mấy năm nhưng còn rất chắc_chắn. Thiết_kế nhà bê_tông cốt thép chắc_chắn, thiết_kế hiện_đại, nhà bao_gồm: phòng khách, bếp, 04 phòng ngủ, phòng thờ, sân phơi + Sổ_đỏ chính chủ, pháp_lý rõ_ràng, sẵn sang giao_dịch + Giá 5.3 tỷ, có thương_lượng cho khách có thiện trí Liên_Hệ: Thanh_Tùng BĐS Ba_Đình: 0912142902, Quý_khách gọi ngay để được tư_vấn nhiệt_tình và xem nhà miễn_phí Cần_Bán, ở ngay, ngõ thoáng, nhà còn mới, Ngõ thông, Đào_Tấn, Ba_Đình Liên_hệ: 09121*** hiện số Loại tin rao Bán nhà_riêng Pháp_lý Sổ Hồng_Số phòng 0 Đường vào 0 m Mặt_tiền 0 m Số tầng 4 Số toilet 0 Thông_tin liên_hệ Tên liên_hệ: